<a href="https://colab.research.google.com/github/bernardooduarte/gestao-projetos-machine-learning/blob/main/C%C3%B3pia_de_03_Machine_Learning_Regressa%CC%83o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo
Prever o valor de venda de uma casa

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [29]:
from google.colab import drive

# Permite acesso aos arquivos
drive.mount('/content/drive', force_remount=True)

# Caminho da pasta que contém as bases de dados.
# É necessário que o arquivo esteja em seu drive, ou crie um atalho da parta compartilhada no seu drive
drive_path = "/content/drive/MyDrive/Gestão de projetos em IA - Translator 5 - Aprofundamentos em Machine Learning/bases/"

Mounted at /content/drive


In [30]:
df = pd.read_csv(drive_path+ 'casas.csv')
df.head()

,moradia_zoneamento,rua_tipo,lote_formato,lote_config,terreno_contorno,moradia_estilo,geral_qualidade,geral_condicao,construcao_ano,remodelacao_ano,primeiro_andar_area,segundo_andar_area,qtd_banheiros,qtd_quartos,qtd_comodos,venda_valor
0,Residencial baixa densidade,pavimentada,levemente irregular,lote de esquina,aproximadamente plana/nivelada,um andar,acima da média,média,1960,1960,1656,0,1,3,7,215000
1,Residencial alta densidade,pavimentada,regular,lote interno,aproximadamente plana/nivelada,um andar,média,acima da média,1961,1961,896,0,1,2,5,105000
2,Residencial baixa densidade,pavimentada,levemente irregular,lote de esquina,aproximadamente plana/nivelada,um andar,acima da média,acima da média,1958,1958,1329,0,1,3,6,172000
3,Residencial baixa densidade,pavimentada,regular,lote de esquina,aproximadamente plana/nivelada,um andar,boa,média,1968,1968,2110,0,2,3,8,244000
4,Residencial baixa densidade,pavimentada,levemente irregular,lote interno,aproximadamente plana/nivelada,dois andares,média,média,1997,1998,928,701,2,3,6,189900


A base de dados possui 2930 entradas e 16 colunas.

1. `moradia_zoneamento`: Tipo de zoneamento residencial (categórica).
2. `rua_tipo`: Tipo de pavimentação da rua (categórica).
3. `lote_formato`: Formato do lote (categórica).
4. `lote_config`: Configuração do lote (categórica).
5. `terreno_contorno`: Contorno do terreno (categórica).
6. `moradia_estilo`: Estilo da moradia (categórica).
7. `geral_qualidade`: Qualidade geral da moradia (categórica).
8. `geral_condicao`: Condição geral da moradia (categórica).
9. `construcao_ano`: Ano de construção (numérica).
10. `remodelacao_ano`: Ano de remodelação (numérica).
11. `primeiro_andar_area`: Área do primeiro andar (numérica).
12. `segundo_andar_area`: Área do segundo andar (numérica).
13. `qtd_banheiros`: Quantidade de banheiros (numérica).
14. `qtd_quartos`: Quantidade de quartos (numérica).
15. `qtd_comodos`: Quantidade de cômodos (numérica).
16. `venda_valor`: Valor de venda (numérica).

A variável `venda_valor` é a nossa target, o que queremos prever.

# Treinamento de Modelos

In [31]:
# Divisão dos dados em features (X) e target (y)
X = df.drop('venda_valor', axis=1)
y = df['venda_valor']

In [32]:
# Dividindo os dados em treino, validação e teste
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [33]:
print(X_train.shape)
print(X_temp.shape)
print(X_val.shape)
print(X_test.shape)

(2051, 15)
(879, 15)
(439, 15)
(440, 15)


In [34]:
# Identificando colunas categóricas e numéricas
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]

In [35]:
categorical_cols

['moradia_zoneamento',
 'rua_tipo',
 'lote_formato',
 'lote_config',
 'terreno_contorno',
 'moradia_estilo',
 'geral_qualidade',
 'geral_condicao']

In [36]:
numerical_cols

['construcao_ano',
 'remodelacao_ano',
 'primeiro_andar_area',
 'segundo_andar_area',
 'qtd_banheiros',
 'qtd_quartos',
 'qtd_comodos']

Instanciando a regressão linear

In [37]:
linear = LinearRegression()

Apenas duas variáveis

In [38]:
linear.fit(X_train[["qtd_comodos","construcao_ano"]], y_train)

LinearRegression()

In [39]:
linear.coef_

array([21956.81083958,  1305.19493661])

In [40]:
linear.intercept_

-2533712.632575573

In [41]:
-2533712+21957*qtd_comodos+1305*construcao_ano

NameError: name 'qtd_comodos' is not defined

In [42]:
prev = linear.predict(X_val[["qtd_comodos","construcao_ano"]])

Calculando as métricas

In [43]:
mean_squared_error(y_val, prev)

3397713118.393261

In [45]:
mean_absolute_error(y_val, prev)

40820.67520328996

In [44]:
r2_score(y_val, prev)

0.5038219407178011

Utilizando todas as colunas numéricas

In [46]:
linear = LinearRegression()
linear.fit(X_train[numerical_cols], y_train)
prev = linear.predict(X_val[numerical_cols])
print(mean_squared_error(y_val, prev))
print(mean_absolute_error(y_val, prev))
print(r2_score(y_val, prev))

1429773760.5041854
26288.353739205486
0.7912059244027477


In [47]:
linear.coef_

array([   723.67100621,    567.60091218,    128.00619234,     92.22210889,
        -7309.38439888, -13082.26603612,     56.96937144])

In [48]:
numerical_cols

['construcao_ano',
 'remodelacao_ano',
 'primeiro_andar_area',
 'segundo_andar_area',
 'qtd_banheiros',
 'qtd_quartos',
 'qtd_comodos']

Criando dummies para as categóricas

In [49]:
X_train[categorical_cols]

,moradia_zoneamento,rua_tipo,lote_formato,lote_config,terreno_contorno,moradia_estilo,geral_qualidade,geral_condicao
2210,Residencial alta densidade,pavimentada,regular,lote interno,encosta,dois andares,média,acima da média
782,Residencial baixa densidade,pavimentada,levemente irregular,lote interno,depressão,um andar,média,muito boa
2310,Residencial baixa densidade,pavimentada,levemente irregular,lote interno,aproximadamente plana/nivelada,dois andares,acima da média,média
299,Residencial média densidade,pavimentada,levemente irregular,lote interno,encosta,um andar,muito boa,média
2423,Residencial média densidade,pavimentada,regular,lote interno,aproximadamente plana/nivelada,um andar,boa,média
...,...,...,...,...,...,...,...,...
1638,Residencial baixa densidade,pavimentada,levemente irregular,fachada em dois lados da propriedade,aproximadamente plana/nivelada,um andar,média,média
1095,Residencial baixa densidade,pavimentada,regular,lote interno,aproximadamente plana/nivelada,dois andares,boa,média
1130,Residencial vila flutuante,pavimentada,levemente irregular,lote interno,aproximadamente plana/nivelada,dois andares,acima da média,média
1294,Residencial média densidade,pavimentada,regular,lote de esquina,aproximadamente plana/nivelada,um andar e meio (segundo nível terminado),média,média


In [50]:
X_train.rua_tipo.value_counts()

,count
rua_tipo,
pavimentada,2041
cascalho,10


In [51]:
one_hot_enc = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'),
    categorical_cols),
    remainder='passthrough')

In [52]:
X_train = one_hot_enc.fit_transform(X_train)
X_val = one_hot_enc.transform(X_val)
X_test = one_hot_enc.transform(X_test)

In [53]:
pd.DataFrame.sparse.from_spmatrix(X_train, columns=one_hot_enc.get_feature_names_out()).head()

,onehotencoder__moradia_zoneamento_Agricultura,onehotencoder__moradia_zoneamento_Comercial,onehotencoder__moradia_zoneamento_Industrial,onehotencoder__moradia_zoneamento_Residencial alta densidade,onehotencoder__moradia_zoneamento_Residencial baixa densidade,onehotencoder__moradia_zoneamento_Residencial média densidade,onehotencoder__moradia_zoneamento_Residencial vila flutuante,onehotencoder__rua_tipo_cascalho,onehotencoder__rua_tipo_pavimentada,onehotencoder__lote_formato_irregular,...,onehotencoder__geral_condicao_média,onehotencoder__geral_condicao_regular,onehotencoder__geral_condicao_ruim,remainder__construcao_ano,remainder__remodelacao_ano,remainder__primeiro_andar_area,remainder__segundo_andar_area,remainder__qtd_banheiros,remainder__qtd_quartos,remainder__qtd_comodos
0,0,0,0,1.0,0,0,0,0,1.0,0,...,0,0,0,1949.0,1950.0,1001.0,1001.0,2.0,4.0,8.0
1,0,0,0,0,1.0,0,0,0,1.0,0,...,0,0,0,1968.0,2003.0,1473.0,0,1.0,1.0,5.0
2,0,0,0,0,1.0,0,0,0,1.0,0,...,1.0,0,0,1997.0,1998.0,813.0,712.0,2.0,3.0,6.0
3,0,0,0,0,0,1.0,0,0,1.0,0,...,1.0,0,0,1989.0,1989.0,1191.0,0,2.0,2.0,5.0
4,0,0,0,0,0,1.0,0,0,1.0,0,...,1.0,0,0,2004.0,2004.0,1414.0,0,2.0,2.0,6.0


Utilizando todas as colunas

In [54]:
print(X_train.shape)

(2051, 56)


In [55]:
linear = LinearRegression()
linear.fit(X_train, y_train)
prev = linear.predict(X_val)
print(mean_squared_error(y_val, prev))
print(mean_absolute_error(y_val, prev))
print(r2_score(y_val, prev))

810244167.7394401
19864.296269518316
0.8816776565045054


In [56]:
linear.coef_

array([-4.01478594e+04,  7.89897925e+02, -5.29452049e+03,  6.69508990e+03,
        1.89175104e+04,  4.93599137e+03,  1.41038903e+04,  3.66931867e+03,
       -3.66931867e+03, -2.71449678e+04,  4.40833385e+03,  2.09836228e+04,
        1.75301110e+03, -7.36634097e+03,  3.99240834e+03, -2.40461414e+03,
       -1.11839861e+03,  6.89694538e+03, -3.99494584e+03, -2.15139859e+04,
        1.25976071e+04,  1.29113246e+04, -1.20387901e+04, -1.07194679e+04,
        6.70144483e+02,  1.08288890e+04, -4.63587777e+03,  6.68027234e+03,
        1.34572344e+04, -4.24240429e+03,  1.30318181e+05, -4.70565074e+04,
       -2.66181159e+04, -1.14647903e+04,  1.02374981e+05,  2.82493157e+04,
       -4.96271403e+04, -3.73819170e+04, -4.52452474e+04, -4.35487598e+04,
       -4.55200679e+03,  1.14526858e+04,  1.90008907e+04,  2.37733369e+04,
        2.20985150e+04, -3.48916016e+04,  5.79437049e+03, -2.25393377e+04,
       -2.01368528e+04,  4.35107666e+02,  1.78747003e+02,  7.73519114e+01,
        8.07598066e+01, -